# How to find answers using jupyter?

Include this header script in your notebooks to find more answers.

How to use inheritence in jupyter notebooks?

How to make a class out of jupter notebooks?

How to use jupyter notebooks as services?



In [1]:
// initialize

// TLDR; What do we end up with?
var importNotebook = (notebook) => Promise.resolve({});
var getCells = ((notebook, types = ['code']) => Promise.resolve([]));
var runAllInNEL = (cells) => Promise.resolve({values: [], session: {}});
var assignResults = (results, filename) => results;

// provide a function for importing any notebook as a module and executing it
var importNotebook = (notebook) => {
    return getCells(notebook, 'code')
            .then(cells => cells.map(c => c.source.join('').trim()))
            .then(cells => runAllInNEL(cells))
            .then((results) => assignResults(results, notebook));
};

// $$.done(); for our dumb parser



What do I need to have installed?

How to sanity check the environment?


In [13]:
// initialize

var execSync = require('child_process').execSync;
var deps = ['npm install', 'JSONStream', 'nel', 'path', 'fs', 'vm'];
var assign = v => Object.assign(global[v] = require(v));
try {
    require.resolve('JSONStream');
    require.resolve('nel');
} catch (e) {
    execSync(deps.slice(0, 3).join(' '), {stdio: 'inherit'});
}
deps.slice(1).forEach(assign);

// $$.done() for our dumb parser


How to stream json using a {match} function?

In [4]:
// initialize

(streamJson = (file, parse, match = ((c) => true)) => {
    var file = fs.createReadStream(file)
        .pipe(JSONStream.parse(parse));
    file.on('data', (m) => match(m));
    // create a promise out of this stream
    return new Promise((resolve, reject) => {
        file.on('error', e => reject(e, file));
        file.on('close', () => resolve(file))
    });
});
// $$.done() for our dumb parser




How to parse cells of {types} from a notebook?



In [7]:
// initialize

// How to use getCells?
(getCells = (notebook, types = ['code']) => {
    var cells = [];
    return streamJson(notebook, 'cells.*', (match) => {
        if(types.indexOf(match.cell_type) > -1) {
            cells[cells.length] = match;
        }
    }).then((file) => cells);
});

// $$.done() for our dumb parser


How to run all promises sequentially?



In [6]:
// initialize

var runAllPromises = (promises) => {
    return promises.reduce((promise, func) => {
        return promise.then(result => {
            return (new Promise(func))
                .then(Array.prototype.concat.bind(result));
        });
    },Promise.resolve([]));
};

// How to create VM promises out of the cell content?
(runAllInNEL = (cells, sess) => {
    if(typeof sess === 'undefined') {
        sess = new nel.Session();
    }
    return runAllPromises(
        cells.map(
            (cell) =>
            (resolve) =>
            sess.execute(cell, {
                onSuccess: (o) => resolve(o),
                onError: (o) => resolve(o),
                beforeRun: () => {}})
        )).then((r) => ({values: r, session: sess}));
});
// $$.done() for our dumb parser



In [1]:
// initialize
if (typeof imported == 'undefined') { var imported = {}; }

// display a heading when new notebooks are imported
var logIm = (cells, filename) => {
    return 'Running ' + filename  
                + ' with ' + cells.length + ' cells';
    console.log();
    return ;
};

function wrapCallback (func, session) {
    return (a1,a2,a3,a4,a5) => {
        var exec = '$$.async(); Promise.resolve(' 
        + func + '.apply(this,' 
        + JSON.stringify([a1,a2,a3,a4,a5]) 
        + ')).then(r => $$.mime({"text/json": r}));';
        return runAllInNEL([exec], session)
            .then(r => {
                var value = r['values'][0]['mime']['text/json'];
                return value;
        });
    };
}

// cache results when notebooks are processed
(assignResults = (results, notebook) => {
    var filename = path.basename(notebook);
    var session = results['session'];
    var results = results['values'].reduce((obj, r, i) => {
        if (typeof r === 'object' && typeof r['mime'] !== 'undefined'
            && typeof r['mime']['text/plain'] !== 'undefined') {
            var re = (/\[Function:\s*(.*?)\]/ig);
            var name = re.exec(r['mime']['text/plain']);
            if(name !== null) {
                obj[filename+'['+i+']'] = wrapCallback(name[1], session);
                obj[name[1]] = obj[filename+'['+i+']'];
            }
        }
        return obj;
    }, {});
    Object.assign(global, results);
    imported[filename] = Promise.resolve(results);
    return results;
});

// $$.done() for our dumb parser


How to import a {notebook}?

How to test the import feature?


In [4]:

// initialize
// What does import return?
imported['How to find answers.ipynb'] = Promise.resolve({
    'How to find answers.ipynb[0]': importNotebook,
    'How to find answers.ipynb[1]': getCells
});

// How to test if a notebook has already been imported?
var oldImport = importNotebook;
(importNotebook = (notebook) => {
    var filename = path.basename(notebook);
    if(typeof notebook === 'undefined') {
        return Promise.resolve({});
    }
    if (typeof imported[filename] !== 'undefined') {
        console.log('Already imported');
        console.log(Object.keys(imported));
        return imported[filename];
    }
    return oldImport(notebook);
});

// $$.done() for our dumb parser


These are some other useful notebooks you may want to import.

How to train a chatbot using jupyter services?

What is code completion for jupter notebooks?

